In [ ]:
!pip install emoji
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 175 kB 14.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=62a877a83433cc91aa4a155a4b2e09cb2291bd9290c31cc9e9de6640246066e8
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 3.8 MB 14.8 MB/s 
     |████████████████████████████████| 67 kB 7.1 MB/s 
     |████████████████████████████████| 596 kB 75.2 MB/s 
     |████████████████████████████████| 6.5 MB 61.6 MB/s 
     |████████████████████████████████| 895 kB 66.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 14.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import emoji
import re
RANDOM_SEED = 123

In [ ]:
emo_data2 = pd.read_csv('/content/drive/MyDrive/Emogen/cleaned_emogen_mask_full.csv')
emo_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8819812 entries, 0 to 8819811
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Datetime       object
 1   Text           object
 2   Label          object
 3   Unmasked_Text  object
dtypes: object(4)
memory usage: 269.2+ MB


In [ ]:
min(emo_data2.groupby('Label').size())

70827

In [ ]:
datasets = []
for emoji in emo_data2.Label.unique():
  temp = emo_data2.query('Label == @emoji').sample(70000)
  datasets += [temp]

In [ ]:
combined_60 = pd.concat(datasets).reset_index(drop = True)
combined_60.head()

,Datetime,Text,Label,Unmasked_Text
0,2022-03-18 18:53:18+00:00,the accuracy&gt;&gt;&gt;&gt;:face_with_tears_o...,😭,the accuracy&gt;&gt;&gt;&gt;😂😂 😭
1,2022-03-18 15:57:40+00:00,i don’t think i can manage to get artifacts th...,😭,i don’t think i can manage to get artifacts th...
2,2022-03-18 16:22:44+00:00,i hesitated to buy a pc and now it’s gone [...,😭,i hesitated to buy a pc and now it’s gone 😭 😭
3,2022-03-18 19:40:06+00:00,:rolling_on_the_floor_laughing::rolling_on_the...,😭,🤣🤣🤣\nmy job tired of me dressing like an hobot...
4,2022-03-18 14:45:18+00:00,mouloud sghir [MASK] wtf,😭,mouloud sghir 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭...


In [ ]:
combined_60.to_csv('/content/drive/MyDrive/Emogen/emogen_mask_70.csv', index = False )

In [ ]:
emo_data = pd.read_csv('/content/drive/MyDrive/Emogen/emogen_mask_70.csv')
emo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3570000 entries, 0 to 3569999
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Datetime       object
 1   Text           object
 2   Label          object
 3   Unmasked_Text  object
dtypes: object(4)
memory usage: 108.9+ MB


In [ ]:
def text_clean(x,emojis):
    x = re.sub(r"{}".format('MASK'), emojis, x) 
    x = re.sub(r'\[', '', x)
    x = re.sub(r'\]', '', x)
    x = emoji.emojize(x)
    return x

In [ ]:
def text_clean3(text):
    x = emoji.emojize(text)
    return x

In [ ]:
def text_clean2(text):
    b = filter(lambda x:x[0]!='@', text.split())
    b = " ".join(b)
    return b

In [ ]:
def replace_hash(x):
   x = re.sub(r'#([^\s]+)' , r'\1', x)
   x = re.sub('(www\.[\s]+)', '', x)
   x = x.lower()
   return x

In [ ]:
def one_mask(x):
  return x.replace('[MASK]', '', x.count('[MASK]')-1)

In [ ]:
emo_data = emo_data[emo_data.Unmasked_Text.str.split().apply(len) < 30]
emo_data = emo_data[emo_data.Unmasked_Text.str.split().apply(len) > 2]
emo_data = emo_data.dropna()
emo_data = emo_data.query('Label != ""')
emo_data = emo_data.query('Text != ""')
emo_data = emo_data[emo_data.Text.str.contains('[MASK]')]
emo_data[-emo_data.Unmasked_Text.str.contains('[MASK]')]
#for index, emo in enumerate(emo_data.Label):
#  emo_data.Unmasked_Text.iloc[index] = text_clean(emo_data.Unmasked_Text.iloc[index], emo)
emo_data['Text'] = emo_data.Text.apply(replace_hash)
emo_data['Unmasked_Text'] = emo_data.Unmasked_Text.apply(replace_hash)
emo_data['Text'] = emo_data.Text.apply(text_clean3)
emo_data['Text'] = emo_data.Text.apply(text_clean2)
emo_data['Text'] = emo_data.Text.apply(one_mask)
emo_data['Unmasked_Text'] = emo_data.Unmasked_Text.apply(text_clean2)
emo_data = emo_data[-emo_data.Unmasked_Text.str.contains('[MASK]')]
emo_data.head()

,Datetime,Text,Label,Unmasked_Text
0,2022-03-18 18:53:18+00:00,the accuracy&gt;&gt;&gt;&gt;😂😂 [mask],😭,the accuracy&gt;&gt;&gt;&gt;😂😂 😭
2,2022-03-18 16:22:44+00:00,i hesitated to buy a pc and now it’s gone [mask],😭,i hesitated to buy a pc and now it’s gone 😭 😭
4,2022-03-18 14:45:18+00:00,mouloud sghir [mask] wtf,😭,mouloud sghir 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 wtf
5,2022-03-18 19:37:43+00:00,also nobody gonna stop saying the word himbo b...,😭,also nobody gonna stop saying the word himbo b...
6,2022-03-18 15:42:44+00:00,i just wanna know what channel the game gone b...,😭,i just wanna know what channel the game gone b...


In [ ]:
emo_data = emo_data[-emo_data.Text.str.contains('&')]
emo_data = emo_data[-emo_data.Text.str.contains('/')]
emo_data = emo_data[-emo_data.Text.str.contains(" \ ")]
emo_data = emo_data[-emo_data.Text.str.contains('{')]
emo_data = emo_data[-emo_data.Text.str.contains('}')]
emo_data = emo_data[-emo_data.Text.str.contains('_')]
emo_data.head()

,Datetime,Text,Label,Unmasked_Text
2,2022-03-18 16:22:44+00:00,i hesitated to buy a pc and now it’s gone [mask],😭,i hesitated to buy a pc and now it’s gone 😭 😭
4,2022-03-18 14:45:18+00:00,mouloud sghir [mask] wtf,😭,mouloud sghir 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 wtf
5,2022-03-18 19:37:43+00:00,also nobody gonna stop saying the word himbo b...,😭,also nobody gonna stop saying the word himbo b...
6,2022-03-18 15:42:44+00:00,i just wanna know what channel the game gone b...,😭,i just wanna know what channel the game gone b...
7,2022-03-18 14:43:22+00:00,lmfaoooooo [mask] please seek help x,😭,lmfaoooooo 😭 😭 😭 please seek help x


In [ ]:

emo_data = emo_data.dropna()
emo_data = emo_data.reset_index(drop= True)
emo_data.to_csv('/content/drive/MyDrive/Emogen/emogen_mask_70.csv', index = False)

In [ ]:
emo_data = pd.read_csv('/content/drive/MyDrive/Emogen/emogen_mask_70.csv')